In [1]:
# dependencies
import ast          # new library: abstract syntax trees!
import pandas as pd
import pymongo
import re           # new library: regex!b

In [2]:
# connect to mongo database "DataMate"
conn = 'mongodb://localhost:27017'
cli = pymongo.MongoClient(conn)
db = cli.DataMate

In [3]:
# read JSON from file
strPath = "../scraping/test_output.txt"
with open(strPath, encoding='utf-8') as file:
    dictProfile = ast.literal_eval(file.read())

In [4]:
# write Profiles to DataMate db table "Profile"
db.Profile.insert_many([dictProfile])

In [5]:
# the CountWords function breaks "hi...let's" into "hi" and "let's"
def CountWords(strChunk, dict):
    lst = re.split(r"[^a-zA-Z0-9']+", strChunk) # split on all chars except letters, numbers, and apostrophes
    for strWord in lst:
        if len(strWord) > 0:
            dict = CountWord(strWord, dict)
    return dict

In [6]:
# the CountWord function logs word counts in dictAboutMe
def CountWord(strWord, dict):
    if strWord in dict:    # have we already logged this word?
        dict[strWord] += 1 # if yes: increment word's count
    else:
        dict[strWord] = 1  # if no: add row to dict
    return dict

In [7]:
# instantiate dictionary for "About Me" field's word frequency
dictAboutMe = {}

In [8]:
# populate "About Me" dictionary
for strKey, strVal in dictProfile.items():
    if(strKey != '_id'):
        for strChunk in strVal['about_me_split']:
            dictAboutMe = CountWords(strChunk.lower(), dictAboutMe)

In [9]:
# convert to dataframe and sort
dfAboutMe = pd.DataFrame.from_dict(dictAboutMe, orient='index', columns=['count'])
dfAboutMe = dfAboutMe.sort_values(['count'], ascending=False)

In [10]:
# build list of dull words
lstDull = ['a', 'am', 'and', 'are', 'as', 'at', 'be', 'but', 'for', 'if', 'in', 'is', 'it', 'not', 'of', 'on', 'or', 'out', 'that', 'the', 'to', 'was', 'with']

In [11]:
# remove boring words from About Me
dfAboutMe = dfAboutMe.drop(lstDull)

In [12]:
dfAboutMe

,count
i,386
my,109
have,87
like,83
me,68
you,63
love,52
i'm,46
looking,46
life,41
